In [1]:
import sys
sys.path.append("/home/daria_l/src/MapMet/code/python")

In [2]:
from Segmentation.Cellpose_Retrained import main_function
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import xml
import pandas as pd
import io
import os
import shutil
from Segmentation.Cellpose_Retrained.morphology_functions import DEEPCELL_MEASURES
from Segmentation.Cellpose_Retrained.intensity_functions import mean_intensity, mean_80_intensity

/home/daria_l/anaconda3/envs/cellpose/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from cellpose import models 
model_path = '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_simon/diameter_evaluation/ausschnitte/models/CP_clusters_final'
#model_path = "CPx"
m = models.CellposeModel(model_type = model_path, gpu=True)

def read_channel_names(path):
    
    tiff = tifffile.TiffFile(path)
    omexml_string = tiff.pages[0].description
    root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
    namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
    channel_names = [c.attrib['Name'] for c in channels]
    return channel_names

In [4]:
base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

BM_path = os.path.join(base_path, "BM")
out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/20230202_sc_data")
samples = os.listdir(BM_path)

for s in samples[50:]: 

    lowres_path = os.path.join(BM_path, s, "IF_IMC_processed")
    highres_path = os.path.join(BM_path, s, "IF_cut_highres")

    lowres_image_files = os.listdir(lowres_path)

    for f in lowres_image_files:

        shutil.copyfile(os.path.join(lowres_path, f), os.path.join(out_path, "img", f))

        IMC = tifffile.imread(os.path.join(lowres_path, f))
        I = tifffile.imread(os.path.join(highres_path, f.split("IMC")[0]+"cut.tiff"))

        channel_names = read_channel_names(os.path.join(lowres_path, f))

        masks, masks_d_1px, masks_d_2px, i_features, i_features_d_1px, i_features_d_2px, m_features = main_function(I, 
                                   intensity_image=IMC,
                                   cellpose_net=m,
                                   eval_kwargs={"diameter": 55}, 
                                   refine=True, t=0.12, 
                                   out_sz=None,
                                   extract_intensity_features=True, 
                                   #intensity_function=mean_intensity,
                                   channel_names=channel_names,
                                   extract_morph_features=True,
                                   additional_morphology_functions=DEEPCELL_MEASURES,
                                   debug_msg=False)

        tifffile.imwrite(os.path.join(out_path, "img_vis", f), IMC[-3])
        m_features.to_csv(os.path.join(out_path, "regionprops", f.split(".")[0] + ".csv"))


        tifffile.imwrite(os.path.join(out_path, "masks_no-dil", f), np.expand_dims(masks, axis=(0, 1, 2, 5)))
        tifffile.imwrite(os.path.join(out_path, "masks_vis_no-dil", f), masks)
        i_features.to_csv(os.path.join(out_path, "intensities_no-dil", f.split(".")[0] + ".csv"))

        tifffile.imwrite(os.path.join(out_path, "masks_dil-1px", f), np.expand_dims(masks_d_1px, axis=(0, 1, 2, 5)))
        tifffile.imwrite(os.path.join(out_path, "masks_vis_dil-1px", f), masks_d_1px)
        i_features_d_1px.to_csv(os.path.join(out_path, "intensities_dil-1px", f.split(".")[0] + ".csv"))

        tifffile.imwrite(os.path.join(out_path, "masks_dil-2px", f), np.expand_dims(masks_d_2px, axis=(0, 1, 2, 5)))
        tifffile.imwrite(os.path.join(out_path, "masks_vis_dil-2px", f), masks_d_2px)
        i_features_d_2px.to_csv(os.path.join(out_path, "intensities_dil-2px", f.split(".")[0] + ".csv"))




KeyboardInterrupt: 